Data

Load data

In [1]:
import nbinteract as nbi
import pandas as pd
import datetime as dt
import numpy as np

# %matplotlib qt

In [2]:
print("Importing data")

dateparse = lambda x: pd.to_datetime([x])
df = pd.read_csv("data1.csv",
                 parse_dates=[["Index Dt","Act Arr"],["Index Dt","Act Dep"]],
                )#,header =4)
df.drop(["Arr","Dep"], axis=1, inplace=True)
df.columns = ["Arr","Dep","Train","Station"]

print("Cleaning data")

for i in ["Arr","Dep"]:
    df[i] = pd.to_datetime(df[i],errors='coerce')
df['Date'] = df['Arr'].dt.date
    
df.sort_values(by=['Train','Date', 'Arr'], inplace=True,na_position='first')
df["Dwell"] = df.Dep - df.Arr
df['Runtime'] = df.Arr - df.Dep.shift()
df.loc[df['Runtime'] > pd.Timedelta(.5,unit="D"),"Runtime"] = pd.NaT
df.loc[df['Runtime'] < pd.Timedelta(1,unit="s"),"Runtime"] = pd.NaT
df.reset_index(inplace = True,drop=True)

print("Extracting list data")

values = "Date","Train","Station"
dd = {i:None for i in values}

for key in dd:
    dd[key] = list(df[key].unique())
    dd[key] = [x for x in dd[key] if str(x)!= 'nan']
    dd[key].sort()

print("Data lists extracted")

start_date = min(df['Date'])
end_date = max(df['Date'])
df.set_index(['Train','Station','Date'],inplace = True)
df.sort_values(by=["Train","Date",'Arr'])
print(df.shape)

df.head()

Importing data
Cleaning data
Extracting list data
Data lists extracted
(69924, 4)


Arr                 Dep Dwell Runtime
Train Station Date                                      
701   BFD     NaT  NaT 2018-03-01 06:00:00   NaT     NaT
              NaT  NaT 2018-03-02 06:00:00   NaT     NaT
              NaT  NaT 2018-03-03 06:00:00   NaT     NaT
              NaT  NaT 2018-03-04 06:00:00   NaT     NaT
              NaT  NaT 2018-03-05 06:00:00   NaT     NaT

In [3]:
df.iloc[[87]]

,,,Arr,Dep,Dwell,Runtime
Train,Station,Date,,,,
701,FNO,NaT,NaT,2018-06-07 04:25:00,NaT,NaT


In [4]:
options = {
    'title': "Histogram"
}


def hist_response(TrainNumber,Station,RunType,Unit):
    global df
    out = df.xs(
        (TrainNumber, Station),
        level=[0, 'Station'])[RunType].dt.seconds.sort_index()
    units = {
        "Mins": 60,
        "Seconds": 1,
    }
    out = out/units[Unit]
    return(out)


nbi.hist(
    hist_response,
    TrainNumber = dd["Train"],
    Station = dd["Station"],
    RunType = ["Runtime","Dwell"],
    Unit = ["Mins","Seconds"]
#     options = options
)

In [5]:
# nbi.publish(spec, nb_name, template='full', save_first=True)